### Load Pickle files from all environments

In [ ]:
import pickle
from pathlib import Path

root_module = Path.cwd()
ext = ".pkl"
name = "sparse_pendulum"
load_dir = root_module.parent.joinpath("data/")
file_dir = load_dir.joinpath(name + ext)
data = pickle.load(open(file_dir, "rb"))

### Smoothen Returns and Compute Mean + Confidence Intervals

In [ ]:
import numpy as np
from itertools import product
from scipy.stats import sem

WINDOW_SIZE = 5

def rolling_average(w: int, arr: np.ndarray) -> np.ndarray:
    """
    Expects an array of size (num_points, num_seeds) where we want to smoothen out each individual
    curve by a moving average
    """
    if arr.size == 0:
        return arr
    one_array = np.ones(w) / w
    padded_arr = np.pad(arr, [(w-1, 0), (0,0)], mode='edge')
    return np.apply_along_axis(lambda m: np.convolve(m, one_array, mode='valid'), axis=0, arr=padded_arr)

# Get available methods and noise levels to plot
keys_arr = np.array(list((data["returns"].keys())), dtype=object)
agent_types = np.array(list(set(keys_arr[:, 0])), dtype=tuple)
agent_types = agent_types[agent_types.argsort()]
# rearrange to desire order 
order = [1, 3, 0, 2, 4]
agent_types = agent_types[order].tolist()
noise_std = np.sort(list(set(keys_arr[:, 1])))

data["mean_returns"] = {}
data["ci_returns"] = {}
for type, noise in product(agent_types, noise_std):
    idx = (type, noise)
    returns = data["returns"][idx]
    smoothened_returns = rolling_average(WINDOW_SIZE, returns)
    data["mean_returns"][idx] = np.mean(smoothened_returns, axis=-1)
    data["ci_returns"][idx] = sem(smoothened_returns, axis=-1)

### Plot results

In [ ]:
%matplotlib widget
import matplotlib.pyplot as plt
from ube_mbrl.utils.plot import PARAMS

plt.rcParams.update(PARAMS)
plt.rcParams['axes.formatter.useoffset'] = False

num_cols = len(noise_std)
fig, axes = plt.subplots(nrows=1, ncols=num_cols, figsize=(7.5, 1.5),gridspec_kw={'wspace':0.05,'hspace':0.1})

ax_dict = {}
for idx, ax in zip(noise_std, axes.flatten()):
    ax_dict[idx] = ax 

colors = {}
cmap = plt.get_cmap("tab10")
for i, type in enumerate(agent_types):
    colors[type] = cmap(i)

ep_length = 400

for type, noise in product(agent_types[::-1], noise_std):
    idx = (type, noise)
    if type == "exact_ube_3":
        label = r"\texttt{exact-ube}" + " (ours)"
    elif type == "ensemble":
        label = r"\texttt{ensemble-var}"
    elif type == "none":
        label = r"\texttt{ensemble-mean}"
    else: label = fr"\texttt{{{type}}}"
    ax_dict[noise].plot(
        data["steps"][idx] // ep_length,
        data["mean_returns"][idx], 
        linestyle="-", 
        linewidth=2, 
        label=label, 
        c=colors[type])
    ax_dict[noise].fill_between(
        data["steps"][idx] // ep_length,
        data["mean_returns"][idx] - data["ci_returns"][idx], 
        data["mean_returns"][idx] + data["ci_returns"][idx], 
        alpha=0.2, color=colors[type]
    )

handles, labels = axes[0].get_legend_handles_labels()
axes[0].legend(
    handles[::-1],
    labels[::-1], 
    loc = 'lower center', ncol=5, columnspacing=0.8, bbox_to_anchor=(1.5, -0.6), frameon=False
)

axes[0].set_ylabel("Return")

# Remove y-axis ticklabels of all but the first column
for ax in axes[1:].flatten():
    ax.yaxis.set_ticklabels([])

# Set x axis label
for ax in axes:
    ax.set_xticks([0, 25, 50, 75])
    ax.set_xlabel("Episode")

for noise, ax in ax_dict.items():
    ax.set_title(f"Angle noise std {noise}")
    ax.set_ylim(bottom=-80, top=240)

In [ ]:
# Save figures
import os
import pickle
from pathlib import Path
root_module = Path.cwd()
fig_dir = root_module.parent.joinpath(f"figures/sparse_pendulum_returns.pdf")
fig.savefig(fig_dir, bbox_inches="tight", transparent=False)

# License

>Copyright (c) 2023 Robert Bosch GmbH
>
>This program is free software: you can redistribute it and/or modify <br>
>it under the terms of the GNU Affero General Public License as published<br>
>by the Free Software Foundation, either version 3 of the License, or<br>
>(at your option) any later version.<br>
>
>This program is distributed in the hope that it will be useful,<br>
>but WITHOUT ANY WARRANTY; without even the implied warranty of<br>
>MERCHANTABILITY or FITNESS FOR A PARTICULAR PURPOSE.  See the<br>
>GNU Affero General Public License for more details.<br>
>
>You should have received a copy of the GNU Affero General Public License<br>
>along with this program.  If not, see <https://www.gnu.org/licenses/>.